In [ ]:
from __future__ import print_function
import sys
sys.path.append("..")
from datasets.es_imagenet import ESImagenet_Dataset
import LIAF
from LIAFnet.LIAFResNet import *
import torch.distributed as dist 
import torch.nn as nn
import argparse, pickle, torch, time, os,sys
from importlib import import_module


##################### Step1. Env Preparation #####################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


##################### Step2. load in dataset #####################

modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
workpath = os.path.abspath(os.getcwd())

num_epochs = config.num_epochs
batch_size = config.batch_size
timeWindows = config.timeWindows
epoch = 0
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
training_iter = 0
start_epoch = 0
acc_record = list([])
loss_train_record = list([])
loss_test_record = list([])

batch_size = 1

test_dataset = ESImagenet_Dataset(mode='test',data_set_path='/Users/lyh/papers/ES-ImageNet-Val/')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8,pin_memory=True)

##################### Step3. establish module #####################

snn = LIAFResNet(config)
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))
snn = LIAFResNet(config)
snn.to(device)

##########################################################
# load in
##########################################################
print('using uniformed init')
pretrain_path = '../pretrained_model/LIAFResNet18-Acc52.pkl'
checkpoint = torch.load(pretrain_path, map_location=torch.device('cpu'))
snn.load_state_dict(checkpoint)


################step4. training and validation ################

def val(snn,test_loader,test_dataset,batch_size,epoch):
    snn.eval()
    correct = 0
    total = 0
    predicted = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            if ((batch_idx+1)<=len(test_dataset)//batch_size):
                try:
                    targets=targets.view(batch_size)#tiny bug
                    outputs = snn(inputs.type(LIAF.dtype))
                    _ , predicted = outputs.cpu().max(1)
                    total += float(targets.size(0))
                    correct += float(predicted.eq(targets).sum())
                    print(batch_idx,'/',len(test_dataset)/batch_size)
                except:
                    print('sth. wrong')
                    print('val_error:',batch_idx, end='')
                    print('taret_size:',targets.size())
    acc = 100. * float(correct) / float(total)
    return acc

print('start validation')
acc = val(snn,test_loader,test_dataset,batch_size,epoch=-1)
print('acc:',acc)




Total number of paramerters in networks is 11693233  
using uniformed init
start validation


In [ ]:
print('acc:',acc)